## Preprocessing

In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dla-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [ ]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN','NAME'])

In [ ]:
# Determine the number of unique values in each column.
application_df.apply(lambda x: len(x.unique()))

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [ ]:
# Look at APPLICATION_TYPE value counts for binning
type_counts=application_df['APPLICATION_TYPE'].value_counts()
type_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [ ]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace=[type_counts.index[i] for i in range(len(type_counts)) if type_counts[i]<500]

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [ ]:
# Look at CLASSIFICATION value counts for binning
class_counts=application_df['CLASSIFICATION'].value_counts()
class_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [ ]:
# You may find it helpful to look at CLASSIFICATION value counts >1
class_counts2 = class_counts.loc[lambda x : x>1]
class_counts2

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [ ]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`

classifications_to_replace=class_counts.loc[lambda x : x<1000].index

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [ ]:
# Convert categorical data to numeric with `pd.get_dummies`
dummies = pd.get_dummies(application_df[["APPLICATION_TYPE","AFFILIATION","CLASSIFICATION","USE_CASE","ORGANIZATION","INCOME_AMT","SPECIAL_CONSIDERATIONS"]])

# Concatenate the dummies to original dataframe
applications_cleaned_df = pd.concat([application_df, dummies], axis=1)

# drop the values
applications_cleaned_df=applications_cleaned_df.drop(["APPLICATION_TYPE","AFFILIATION","CLASSIFICATION","USE_CASE","ORGANIZATION","INCOME_AMT","SPECIAL_CONSIDERATIONS"], axis=1)
applications_cleaned_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [ ]:
# Split our preprocessed data into our features and target arrays
X = applications_cleaned_df.drop(columns=["IS_SUCCESSFUL"])
y = applications_cleaned_df["IS_SUCCESSFUL"]

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
X.shape[1]

43

## Compile, Train and Evaluate the Model

In [ ]:
import math
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

# input dimension
num_features = X.shape[1]

# We require at least as many neurons in the input layer as there are features, so initally we can let

num_neurons_1 = num_features

# For an initial guess we can let the number of neurons in the first hidden layer to be

num_neurons_2 = math.floor(num_features/2)

# And the number of neurons in the second hidden layer to be

num_neurons_3 = math.floor(num_neurons_2)

# initalising model

nn = tf.keras.models.Sequential()

# initially we can just use the sigmoid for all activation functions, though
# the output layer will need a sigmoid activation function since we are doing binary classification

# input layer

nn.add(tf.keras.layers.Dense(units=num_neurons_1, activation="sigmoid", input_dim=num_features))

# First hidden layer
nn.add(tf.keras.layers.Dense(units=num_neurons_2, activation="sigmoid"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=num_neurons_3, activation="sigmoid"))

# Output layer (single neuron)
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))


# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 43)                1892      
                                                                 
 dense_1 (Dense)             (None, 21)                924       
                                                                 
 dense_2 (Dense)             (None, 21)                462       
                                                                 
 dense_3 (Dense)             (None, 1)                 22        
                                                                 
Total params: 3300 (12.89 KB)
Trainable params: 3300 (12.89 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [ ]:
# Train the model

import keras as keras
callbacks = keras.callbacks.ModelCheckpoint('weights{epoch:08d}.h5',
                                     save_weights_only=True, period=5)
fit_model = nn.fit(X_train_scaled, y_train, epochs=50,callbacks=callbacks)

Epoch 1/50
804/804 [==============================] - 6s 5ms/step - loss: 0.6051 - accuracy: 0.6866
Epoch 2/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5713 - accuracy: 0.7280
Epoch 3/50
804/804 [==============================] - 3s 4ms/step - loss: 0.5659 - accuracy: 0.7283
Epoch 4/50
804/804 [==============================] - 6s 7ms/step - loss: 0.5614 - accuracy: 0.7290
Epoch 5/50
804/804 [==============================] - 6s 7ms/step - loss: 0.5587 - accuracy: 0.7303
Epoch 6/50
804/804 [==============================] - 5s 7ms/step - loss: 0.5566 - accuracy: 0.7320
Epoch 7/50
804/804 [==============================] - 6s 7ms/step - loss: 0.5553 - accuracy: 0.7322
Epoch 8/50
804/804 [==============================] - 4s 5ms/step - loss: 0.5542 - accuracy: 0.7318
Epoch 9/50
804/804 [==============================] - 3s 4ms/step - loss: 0.5530 - accuracy: 0.7318
Epoch 10/50
804/804 [==============================] - 4s 5ms/step - loss: 0.5520 - accuracy: 0.7313

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5545 - accuracy: 0.7254 - 343ms/epoch - 1ms/step
Loss: 0.5544561147689819, Accuracy: 0.7253644466400146


In [ ]:
!pip install pyyaml h5py


In [ ]:
# Export our model to HDF5 file
nn.save("initial_model.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
